In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,Trainer,TrainingArguments
!pip install datasets
from datasets import load_dataset,Dataset
from sklearn.model_selection import train_test_split

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import os
# 初始化一个空列表来存储所有的DataFrame
all_dfs = []
data_folder = './data/'
# 遍历data文件夹中的所有文件
for filename in os.listdir(data_folder):
    if filename.endswith('.tsv'):  # 假设所有数据集都是CSV格式
        file_path = os.path.join(data_folder, filename)
        # 读取CSV文件并添加到列表中
        df = pd.read_csv(file_path,sep='\t')
        all_dfs.append(df)

# 使用pd.concat()函数将所有DataFrame合并
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df.info()
combined_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42832 entries, 0 to 42831
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   42832 non-null  int64 
 1   text_a  42832 non-null  object
dtypes: int64(1), object(1)
memory usage: 669.4+ KB


,label,text_a
0,2,//@UNIQ-王一博:#致敬疫情前线医护人员#你们辛苦了！愿平安！[心]
1,4,幸亏我爸妈听话//@北国佳人李春姬:令人窒息
2,2,我大河南！！！！[good][good][good][good][good][good]
3,2,#疫情地图#鸡蛋挺住#青海祈福# ? ??
4,0,肏//@马睿_Ray:[doge]//@Erlang:[doge]//@沈沉舟:那是2003...


In [5]:
data = combined_df
texts = data['text_a'].tolist()  # 文本列
labels = data['label'].tolist()  # 标签列

In [6]:
# 拆分为训练集和验证集
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)
# 加载预训练的中文分词器
tokenizer = AutoTokenizer.from_pretrained("hfl/minirbt-h288")

# 数据预处理函数
def preprocess_data(texts, labels, tokenizer, max_length=128):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
    return encodings

# 对训练集和验证集进行编码
train_encodings = preprocess_data(train_texts, train_labels, tokenizer)
val_encodings = preprocess_data(val_texts, val_labels, tokenizer)

# 将数据转换为 Dataset 格式，供 Trainer 使用
train_dataset = Dataset.from_dict({**train_encodings, 'label': train_labels})
val_dataset = Dataset.from_dict({**val_encodings, 'label': val_labels})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
from transformers import AutoModelForSequenceClassification

# 修改模型为 6 分类输出 只利用前面的特征层
model = AutoModelForSequenceClassification.from_pretrained("hfl/minirbt-h288",num_labels=6,ignore_mismatched_sizes=True)
# 遍历模型的所有参数，并将非连续的张量转换为连续的
for name, param in model.named_parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

pytorch_model.bin:   0%|          | 0.00/49.4M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/minirbt-h288 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

# 加载 accuracy 指标
metric = evaluate.load("accuracy")
# 定义计算指标的函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 训练参数
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy="epoch",
    learning_rate=6e-5,
    warmup_steps=200,
    weight_decay=0.01,
    metric_for_best_model="accuracy"
)

# 使用 Trainer 进行微调
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 开始训练
trainer.train()

# 保存模型
model.save_pretrained('./fine_tuned_bert_chinese')
tokenizer.save_pretrained('./fine_tuned_bert_chinese')

# 验证模型
eval_results = trainer.evaluate()
print(f"Validation accuracy: {eval_results['eval_accuracy']}")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.879100,0.807534,0.720089
2,0.814600,0.775496,0.725575
3,0.632800,0.743943,0.740282
4,0.551600,0.791601,0.738532
5,0.611200,0.775398,0.742033
6,0.472900,0.886615,0.737948
7,0.321900,0.865144,0.741450
8,0.276100,0.939534,0.740049
9,0.387300,0.964274,0.739699
10,0.148100,1.012122,0.741566


Validation accuracy: 0.7420333839150227


In [ ]:
eval_results

{'eval_loss': 0.6654650568962097,
 'eval_runtime': 49.3893,
 'eval_samples_per_second': 153.211,
 'eval_steps_per_second': 2.409,
 'epoch': 5.0}

In [9]:
#处理后的数据对最好的模型重新finetune
import re
data['text_a'] = data['text_a'].str.replace(r'@.*?:', '', regex=True)


In [10]:
texts = data['text_a'].tolist()  # 文本列
labels = data['label'].tolist()  # 标签列
# 拆分为训练集和验证集
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)
# 加载预训练的中文分词器
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_bert_chinese")

# 数据预处理函数
def preprocess_data(texts, labels, tokenizer, max_length=128):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
    return encodings

# 对训练集和验证集进行编码
train_encodings = preprocess_data(train_texts, train_labels, tokenizer)
val_encodings = preprocess_data(val_texts, val_labels, tokenizer)

# 将数据转换为 Dataset 格式，供 Trainer 使用
train_dataset = Dataset.from_dict({**train_encodings, 'label': train_labels})
val_dataset = Dataset.from_dict({**val_encodings, 'label': val_labels})

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("fine_tuned_bert_chinese")

from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

# 加载 accuracy 指标
metric = evaluate.load("accuracy")
# 定义计算指标的函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 训练参数
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy="epoch",
    learning_rate=6e-5,
    warmup_steps=200,
    weight_decay=0.01,
    metric_for_best_model="accuracy"
)

# 使用 Trainer 进行微调
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 开始训练
trainer.train()

# 保存模型
model.save_pretrained('./fine_tuned_bert_chinese_dealed_data')
tokenizer.save_pretrained('./fine_tuned_bert_chinese_dealed_data')

# 验证模型
eval_results = trainer.evaluate()
print(f"Validation accuracy: {eval_results['eval_accuracy']}")


Epoch,Training Loss,Validation Loss,Accuracy
1,0.565200,0.527444,0.822692
2,0.482200,0.566976,0.806000
3,0.363100,0.602486,0.806467
4,0.396600,0.607375,0.806700
5,0.438600,0.642398,0.808218
6,0.216200,0.715258,0.809151
7,0.184500,0.732348,0.807867
8,0.266800,0.798247,0.807167
9,0.237800,0.844636,0.807400
10,0.267100,0.860892,0.808801


Validation accuracy: 0.8226917240574296


In [13]:
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

def convert_safetensors_to_pytorch(input_model_path, output_model_path):
    # 加载 safetensors 模型
    print(f"Loading model from {input_model_path}")
    model = AutoModelForSequenceClassification.from_pretrained(input_model_path, use_safetensors=True)

    # 加载分词器
    tokenizer = AutoTokenizer.from_pretrained(input_model_path)

    # 创建输出目录（如果不存在）
    os.makedirs(output_model_path, exist_ok=True)

    # 直接使用 torch.save 保存模型状态字典
    print(f"Saving model to {output_model_path}")
    torch.save(model.state_dict(), os.path.join(output_model_path, "pytorch_model.bin"))

    # 保存模型配置
    model.config.save_pretrained(output_model_path)

    # 保存分词器
    tokenizer.save_pretrained(output_model_path)

    print(f"Model and tokenizer successfully converted and saved to {output_model_path}")

# 使用函数
input_path = "fine_tuned_bert_chinese_dealed_data"  # 原始 safetensors 模型路径
output_path = "best_sentiment_classification_model" # 新的 PyTorch 模型保存路径

convert_safetensors_to_pytorch(input_path, output_path)

# 验证新保存的模型
print("Verifying the newly saved model...")
loaded_model = AutoModelForSequenceClassification.from_pretrained(output_path)
loaded_tokenizer = AutoTokenizer.from_pretrained(output_path)
print("Model and tokenizer loaded successfully from the new location!")

# 检查保存的文件
print("\nFiles in the output directory:")
for file in os.listdir(output_path):
    print(file)

Loading model from fine_tuned_bert_chinese_dealed_data
Saving model to best_sentiment_classification_model
Model and tokenizer successfully converted and saved to best_sentiment_classification_model
Verifying the newly saved model...
Model and tokenizer loaded successfully from the new location!

Files in the output directory:
special_tokens_map.json
config.json
model.safetensors
pytorch_model.bin
tokenizer.json
tokenizer_config.json
vocab.txt


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!cp -r /content/best_sentiment_classification_model /content/drive/My\ Drive/
